In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as Funct
import sys
import segData
from PIL import Image
from torchvision.transforms import ToPILImage
import torchvision.transforms as t

from Unet import Unet
from Unet import ConvBlock
from Unet import First_ConvBlock
from Unet import Get_Sample
from Unet import Up_Block

import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

In [2]:
torch.cuda.init()
batch_size = 1
test_dataset = segData.DataS("val")
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=8)   

In [3]:
model = torch.load('1_epoch_model.pt')

In [4]:
def iou_r(outputs, label):
    outputs = (outputs > 0.5).type(torch.cuda.FloatTensor)
    label = label.type(torch.cuda.FloatTensor)
    intersection = (outputs * label).type(torch.cuda.FloatTensor)
    ex = 0
    result = []
    for i in range(batch_size):
        ret = (torch.sum(intersection[i][0])/(torch.sum(outputs[i][0])+torch.sum(label[i][0])-torch.sum(intersection[i][0]) + 1e-7))
        result.append(ret.data)

    ret = sum(result)
    return ret

In [5]:
def miou(outputs, label, hyper):
    print(outputs)
    outputs = (outputs.cpu() > hyper).type(torch.cuda.FloatTensor)
    label = label.type(torch.cuda.FloatTensor)
    intersection = (outputs*label).type(torch.cuda.FloatTensor)
    result = []
    for i in range(batch_size):
        ret = (torch.sum(intersection[i][0])/(torch.sum(outputs[i][0])+torch.sum(label[i][0])-torch.sum(intersection[i][0]) + 1e-7))
        result.append(ret.data)

    ret = sum(result)
    return ret

In [6]:
def miou2(outputs, label):
    outputs = outputs.type(torch.cuda.FloatTensor)
    label = label.type(torch.cuda.FloatTensor)
    intersection = (outputs*label).type(torch.cuda.FloatTensor)
    result = []
    for i in range(batch_size):
        ret = (torch.sum(intersection[i][0])/(torch.sum(outputs[i][0])+torch.sum(label[i][0])-torch.sum(intersection[i][0]) + 1e-7))
        result.append(ret.data)

    ret = sum(result)
    return ret

In [7]:
result = 0.0
result2 = 0.0
length = 0
i=0
a = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

for img, label in test_loader:
    temp = 0
    temp2 = 0
    img = Variable(img.cuda())
    output = model(img)
    temp = miou(output,label,0.5)
    temp2 = miou2(output,label)
    result2 += temp2
    result += temp
    torch.cuda.empty_cache()

    
result /= len(test_dataset)
result2 /= len(test_dataset)
print(result)
print(result2)

tensor([[[[ 0.1475,  0.2510,  0.3389,  ...,  0.0279,  0.0253,  0.0273],
          [ 0.2576,  0.3281,  0.3953,  ...,  0.0220,  0.0201,  0.0248],
          [ 0.3339,  0.3873,  0.4233,  ...,  0.0206,  0.0183,  0.0224],
          ...,
          [ 0.2246,  0.2624,  0.3093,  ...,  0.0251,  0.0239,  0.0288],
          [ 0.1658,  0.2053,  0.2620,  ...,  0.0226,  0.0229,  0.0293],
          [ 0.1133,  0.1765,  0.2450,  ...,  0.0272,  0.0300,  0.0297]]]], device='cuda:0')


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1524584710464/work/aten/src/THC/generic/THCStorage.cu:58